# Classification /extraction 
--- 

we should consider only some properties derivated  from items but in this case we can build a vocabulary of property initially for two categories **Culture Representative** and **Culture Agnostic**. 


### How should we chose properties from item of Culture Agnostic and Culture Representative? 
### Training  Phase:

--- 

### Step 1: Building 1 vocabulary of properties associated to Culture Representative and Culture Agnostic 
In this phase, for every sample $ item_{i} $ associated to, we put all its properties  into a set  $S_{P} $
### Step 2: Embedded vectors of properties for every item 
In this case we associate a binary vector for every $item_{i} $ where for every possible property we associate 1 if it has that property $P_k$ otherwise 0.
### Step 3: Compute the centroids for every category C.A C.R, C.E.X
Now, after the preprocessed data we compute the centroid (like mean over every property $P_k$ on all samples) $C_{CA} $ and $C_{CR} $.
### Step 4: We compute the euclidean distance among properties and centroids
for every sample we compute distance among each property and centroids, so we will interpret strong relationship of some  properties wrt culture concepts while for concept more neutrals
### Step 5: Corresponding the similarity to every property with some methods wrt centroids
Every property, will have a weight computed following the importance of distance wrt centroids $C_{C.A} $ and  $ C_{C.R}$ : 
#### Case 1: Kernel Funtion (Kernel Density Estimation)
we will use kernel to give a more flexible weight : $ w_{gauss}(item_{i})=\exp{(-\frac{d_{C}(item_{i})}{2*\sigma^2})} $ .
We can compute **$\sigma$** like a constant such that influences the area of neighbours entities: 
we can compute it as: $\sigma=\frac{1}{\sqrt{2}*mean \space of \space distance}$, can be optimized empirically 
**The Gaussian kernel has the advantage of providing a gradual decrease in weight rather than a linear or inversely proportional decrease, allowing us to assign greater weight to nearby entities without excluding those that are further away.** .
We normalize all weights wrt the sum following weights wrt centroids

### Test Phase:

### Step 1: Compute distance among every element wrt to the both centroids
we compute per every $item_i$  and every feature of kind: $P_{123},P_{2345} $ ecc... the euclidean distance
### Step 2: Compute similarity for every test sample(with kernel approach) for both centroids
this pass helps us to understand in which direction a sample should go to the centroids for both centroids 
### Step 3: Given averaged sum of the importance of feature of samples 
in this step we compute, Culture and Agnostic_score= $ \sum_{fi=1}^{N_feat} item_{fi}* importance \space of  \space features $  the secnond term is measured in previous case
### Step 4 : Then, to emphasize the influence of entities closer to the centroids, we multiply the weighted score by the similarity.
So, ultimately, we multiply the weighted score by similarity to get a total score that is more sensitive to the entity's proximity to the centroids. This helps make a more accurate prediction based on how much the entity resembles the cultural or agnostic centers.

total_culture_score=$ Culture \space and  \space Agnostic_score * similarity \space of \space distance \space of \space test_sample $ repeated for every class score
total_agnostic_score = ... 
### Step 5: Classification based on best result
if total_culture_score > total agnostic_score  -> **Culture Representative**









# Step 0: Building a Vocabulary with all properties of training items

In [14]:
from datasets import load_dataset
import pandas as pd
import torch as t 
import numpy as np
from wikidata.client import Client
from itertools import islice

In [32]:
ds=load_dataset("sapienzanlp/nlp2025_hw1_cultural_dataset")
train_set=train_data=pd.DataFrame(ds["train"])
list_category=set(list(train_set.category))
X_train=train_set.values

print(X_train) # stamp all dataset
#print(X_train.shape) 

[['http://www.wikidata.org/entity/Q32786' '916' '2012 film by M. Mohanan'
  ... 'films' 'film' 'cultural exclusive']
 ['http://www.wikidata.org/entity/Q371' '!!!'
  'American dance-punk band from California' ... 'music' 'musical group'
  'cultural representative']
 ['http://www.wikidata.org/entity/Q3729947' '¡Soborno!'
  'Mort & Phil comic' ... 'comics and anime' 'comics'
  'cultural representative']
 ...
 ['http://www.wikidata.org/entity/Q10779' 'Zwenkau'
  'city in the district of Leipzig, in the Free State of Saxony, Germany'
  ... 'geography' 'city' 'cultural exclusive']
 ['http://www.wikidata.org/entity/Q245296' 'zydeco'
  'music genre evolved in southwest Louisiana which blends blues, rhythm and blues, and music indigenous to the Louisiana Creoles and the Native people of Louisiana'
  ... 'music' 'music genre' 'cultural representative']
 ['http://www.wikidata.org/entity/Q129298' 'Zygmunt Chmielewski'
  'actor (1894-1978)' ... 'performing arts' 'theatrical director'
  'cultural ex

In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]
print(X_train.shape[0])
def extract_sample_from_cat(X,cat):
    l=list()
    for elem in X:
        if elem[4]==cat:
            l.append(elem[0])
    return l
#entity_train=np.zeros(shape=
client = Client()

for cat in list_category:
    list_sample_cat=extract_sample_from_cat(X_train,cat)
    #print("sample category: "+cat)
    #print(len(list_sample_cat))
    set_properties=list()
    for url in list_sample_cat:
        entity_train=extract_entity_id(url)
        item = client.get(entity_train, load=True)
        claim_item_i=item.data.get("claims",{})
       # print(claim_item_i)
        #print(len(claim_item_i))
        set_property_item=set()
        for prop_id,values in islice(claim_item_i.items(),len(claim_item_i)):
            prop_entity = client.get(prop_id, load=True)
            label = prop_entity.label
            #print(f"{prop_id} = {label} ({len(values)} statement{'s' if len(values) != 1 else ''})")
            set_property_item.add(prop_id)

        set_properties.append(set_property_item)
        print(set_properties)
        #set_property_item.clear()
    #print("intersection of sample belongs"+cat,set.intersection(*set_properties))

        #label = prop_entity.labels
        #print(claim_item_i)
    

    

6251
[{'P2671', 'P279', 'P31'}]
[{'P2671', 'P279', 'P31'}, {'P18', 'P373', 'P31', 'P625', 'P17', 'P1417', 'P646', 'P974', 'P1343', 'P131', 'P2043', 'P4614', 'P403', 'P821', 'P469', 'P2053', 'P1566', 'P402', 'P885', 'P2044'}]
[{'P2671', 'P279', 'P31'}, {'P18', 'P373', 'P31', 'P625', 'P17', 'P1417', 'P646', 'P974', 'P1343', 'P131', 'P2043', 'P4614', 'P403', 'P821', 'P469', 'P2053', 'P1566', 'P402', 'P885', 'P2044'}, {'P244', 'P214', 'P590', 'P646', 'P6766', 'P1082', 'P131', 'P774', 'P1937', 'P281', 'P31', 'P625', 'P2044', 'P17', 'P1566', 'P8189'}]
[{'P2671', 'P279', 'P31'}, {'P18', 'P373', 'P31', 'P625', 'P17', 'P1417', 'P646', 'P974', 'P1343', 'P131', 'P2043', 'P4614', 'P403', 'P821', 'P469', 'P2053', 'P1566', 'P402', 'P885', 'P2044'}, {'P244', 'P214', 'P590', 'P646', 'P6766', 'P1082', 'P131', 'P774', 'P1937', 'P281', 'P31', 'P625', 'P2044', 'P17', 'P1566', 'P8189'}, {'P361', 'P646', 'P8408', 'P1282', 'P279', 'P910', 'P4980', 'P1889', 'P227', 'P2670', 'P1687', 'P1014'}]
[{'P2671', 'P279

KeyboardInterrupt: 